First step.

First of all the table with bond information needs to be transform for beеter use.

In [1]:
import pandas as pd

In [3]:
df = pd.read_excel("01_10_2023 в обігу.xlsx", index_col=0, header=1)

In [4]:
# Drop columns wich definitely don't need 
df.drop(columns=["Термін обігу \n(дн.)","Кількість ЦП у \nвипуску, шт.","Стовпець1"], inplace=True)

In [5]:
# Rename for english 
columns_name = ["ISIN",
                "Borrowing currency and type of bond",
                "Date of placement",
                "Interest payment dates",
                "Maturity date",
                "Nominal yield level, %.",
                "Coupon payment amount per bond"
                ]
df.columns = columns_name

In [6]:
# Selec into separate dataframe all bond in UAH currency
df2 = df[df["Borrowing currency and type of bond"].isin(["Гривня"])].copy()

In [7]:
# Let's split each payment for separate row. It will be need in future
df2["Interest payment dates"] = df2["Interest payment dates"].str.split()
df3 = df2.explode("Interest payment dates")

In [8]:
# Reset the index column and remove the currency column. Don't need it anymore
df3.reset_index(drop=True, inplace=True)
df3.drop(columns="Borrowing currency and type of bond", inplace=True)

In [9]:
# After exploring the result, find different data type in the column. Fix it manually. 
# TO DO Try to automate this process as excel's for input may be different
df3["Coupon payment amount per bond"].unique()

array([57.5, 54.75, 58.35, 64.7, 80, '-', 49.95, 70, 61.5, 96.25, 77.5,
       '77,50\n(крім 25.03.2015–76,65)', 97.95, 97.5, 79.2, 98.75, 91.55,
       98.5, 62.599999999999994, 95.9, 48.949999999999996], dtype=object)

In [10]:
# Bonds with "-" in this column aren't avaible for privat investors
df3.drop(df3.loc[df3["Coupon payment amount per bond"]=="-"].index, inplace=True)
#df3[df3["Interest payment dates"] == "25.03.2015"]
df3.loc[df3["Interest payment dates"] == "25.03.2015", ["Coupon payment amount per bond"]] = 76.65
df3.loc[df3["Coupon payment amount per bond"]=="77,50\n(крім 25.03.2015–76,65)", ["Coupon payment amount per bond"]] = 77.50

In [11]:
df3["Coupon payment amount per bond"] = df3["Coupon payment amount per bond"].apply(lambda row: round(row, 2))

In [12]:
df3.dtypes

ISIN                                      object
Date of placement                 datetime64[ns]
Interest payment dates                    object
Maturity date                             object
Nominal yield level, %.                   object
Coupon payment amount per bond           float64
dtype: object

In [13]:
# Will be working with date a lot, so it needs to be in the correct format
df3["Nominal yield level, %."] = df3["Nominal yield level, %."].astype(float)
df3["Maturity date"] = pd.to_datetime(df3["Maturity date"])
df3["Interest payment dates"] = pd.to_datetime(df3["Interest payment dates"], format=r'%d.%m.%Y')
df3.dtypes

ISIN                                      object
Date of placement                 datetime64[ns]
Interest payment dates            datetime64[ns]
Maturity date                     datetime64[ns]
Nominal yield level, %.                  float64
Coupon payment amount per bond           float64
dtype: object

This solution was the easiest way to split the coupon payment for the first and second payments in the year. This will be useful for analitycs part.

In [14]:

df3.loc[df3["Interest payment dates"].dt.month > 6, "Payment"] = "#2 " + df3["Interest payment dates"].dt.year.astype(str)
df3.loc[df3["Interest payment dates"].dt.month <= 6, "Payment"] = "#1 " + df3["Interest payment dates"].dt.year.astype(str)

In [22]:
# This block is trying to get read of time in excel but with no effect.
df3["Date of placement"] = df3["Date of placement"].dt.date
df3["Interest payment dates"] = df3["Interest payment dates"].dt.date
df3["Maturity date"] = df3["Maturity date"].dt.date
df3.dtypes

ISIN                               object
Date of placement                  object
Interest payment dates             object
Maturity date                      object
Nominal yield level, %.           float64
Coupon payment amount per bond    float64
Payment                            object
dtype: object

In the first lunch the result will be saved in new excel file. But in the future the information must be updated from the new input file to the existing file. So, this part also needs to be upgrade. 
Goes to # TO DO

In [15]:
df3.to_excel("ua_bonds.xlsx", sheet_name="Bonds_Info", index=False)